In [2]:
!pip install pandas==0.23.4
!pip install graphviz
!apt-get install graphviz

  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2


'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
conda install -c conda-forge category_encoders 

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\lilyx\Anaconda3

  added / updated specs:
    - category_encoders


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    category_encoders-1.3.0    |             py_0          29 KB  conda-forge
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.6.14               |           py37_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

  category_encoders  conda-forge/noarch::category_encoders-1.3.0-py_0

The following packages will be UPDATED:

  conda                      pkgs/main::conda-4.6.11-py37_0 --> conda-forge::conda-4.6.14-py37_0

The following packages will be SUPERSEDED by a higher-priority channel:




==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base -c defaults conda


'ET' is not recognized as an internal or external command,
operable program or batch file.


In [222]:
%matplotlib inline
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('https://drive.google.com/uc?export=download&id=14ULvX0uOgftTB2s97uS8lIx1nHGQIB0P'), 
                 pd.read_csv('https://drive.google.com/uc?export=download&id=1r441wLr7gKGHGLyPpKauvCuUOU556S2f'))

pop1 = pd.read_csv('tza_pop_adm1.csv')
pop2 = pd.read_csv('tza_pop_adm2.csv')
pop3 = pd.read_csv('tza_pop_adm3.csv')

trainwithpopulation1 = pd.merge(train, pop1, how='outer', left_on = 'region', right_on = 'ADM1_EN')
train = trainwithpopulation1.iloc[0:59400]

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('https://drive.google.com/uc?export=download&id=1wvsYl9hbRbZuIuoaLWCsW_kbcxCdocHz')
sample_submission = pd.read_csv('https://drive.google.com/uc?export=download&id=1kfJewnmhowpUo381oSn3XqsQ6Eto23XV')

testwithpopulation1 = pd.merge(test, pop1, how='outer', left_on = 'region', right_on = 'ADM1_EN')
test = trainwithpopulation1.iloc[0:14358]

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

# Print dataframe shapes
print('train', train.shape)
print('val', val.shape)
print('test', test.shape)

train (47520, 55)
val (11880, 55)
test (14358, 55)


In [223]:
def wrangle(X):
    """Wrangles train, validate, and test sets in the same way"""
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace them with the column mean.
    cols_with_zeros = ['construction_year', 'longitude', 'latitude']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col] = X[col].fillna(X[col].mean())#using the average of a specific district
        
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract year from date_recorded
    X['year_recorded'] = X['date_recorded'].dt.year
    #Extract Never payed in separate column
    X['neverpay'] = X['payment'] == 'never pay'
    
    # quantity & quantity_group are duplicates, so drop one
    X = X.drop(['quantity_group','ADM1_EN', 'date_recorded', 'region_code',
                'district_code','recorded_by','scheme_management', 'scheme_name',
                'extraction_type_group','management','management_group',
                'ADM1_PCODE', 'ADM0_PCODE','TOTAL_BOTH','URBAN_BOTH','population'], axis=1)
    
    
    # for categoricals with missing values, fill with the category 'MISSING'
    categoricals = X.select_dtypes(exclude='number').columns
    for col in categoricals:
        X[col] = X[col].fillna('MISSING')
    
    return X


train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [224]:
#impute
train = train.fillna(0)
val = val.fillna(0)
test = test.fillna(0)

# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target & id
train_features = train.drop(columns=[target, 'id'])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 20].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features



#drop unknowns and missing data
# train['neverpay'] = train[train.payment == 'never pay']

In [225]:
train.shape

(47520, 41)

In [226]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

# Encoder: fit_transform on train, transform on val & test
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

# Scaler: fit_transform on train, transform on val & test
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

In [227]:
X_train_scaled.shape

(47520, 141)

In [228]:
X_train_scaled = pd.DataFrame(X_train_scaled)
X_train_scaled['tojoin'] = 1
X_val_scaled = pd.DataFrame(X_val_scaled)
X_val_scaled['tojoin'] = 1
X_test_scaled = pd.DataFrame(X_test_scaled)
X_test_scaled['tojoin'] = 1

In [229]:
train_numeric_features_to_array = train_features.select_dtypes(include='number')
train_numeric_features_to_array['tojoin'] = 1
val_features = val.drop(columns=[target, 'id'])
val_numeric_features_to_array = val_features.select_dtypes(include='number')
val_numeric_features_to_array['tojoin'] = 1
test_features = test.drop(columns=[target, 'id'])
test_numeric_features_to_array = test_features.select_dtypes(include='number')
test_numeric_features_to_array['tojoin'] = 1

C:\Users\lilyx\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\lilyx\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\lilyx\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [230]:
X_train_scaled.shape

(47520, 142)

In [231]:
test_numeric_features_to_array.head(2)

,amount_tsh,gps_height,longitude,latitude,num_private,construction_year,TOTAL_MALE,TOTAL_FEMA,RURAL_BOTH,RURAL_MALE,URBAN_MALE,URBAN_FEMA,RURAL_FEMA,year_recorded,tojoin
0,6000.00,1390.00,34.94,-9.86,0.00,1999.00,452052.00,489186.00,684890.00,329740.00,122312.00,134036.00,355150.00,2011,1
1,500.00,1703.00,34.64,-9.11,0.00,1978.00,452052.00,489186.00,684890.00,329740.00,122312.00,134036.00,355150.00,2011,1


In [232]:
X_train_scaled.shape, train_numeric_features_to_array.shape

((47520, 142), (47520, 15))

In [233]:
X_train_scaled = pd.merge(X_train_scaled, train_numeric_features_to_array, how='outer', left_index = True, right_index = True)
X_val_scaled = pd.merge(X_val_scaled, val_numeric_features_to_array, how='outer', left_index = True, right_index = True)
X_test_scaled = pd.merge(X_test_scaled, test_numeric_features_to_array, how='outer', left_index = True, right_index = True)

In [234]:
X_train_scaled = X_train_scaled.iloc[0:47520]
X_val_scaled = X_val_scaled.iloc[0:11880]

In [235]:
X_train_scaled = X_train_scaled.dropna()
X_val_scaled = X_val_scaled.dropna()
X_test_scaled = X_test_scaled.dropna()

X_train_scaled = scaler.fit_transform(X_train_scaled)
X_val_scaled = scaler.fit_transform(X_val_scaled)
X_test_scaled = scaler.fit_transform(X_test_scaled)

X_train_scaled.shape, X_val_scaled.shape, X_test_scaled.shape, y_train.shape, y_val.shape

((38100, 157), (2323, 157), (14358, 157), (47520,), (11880,))

In [236]:
from sklearn.tree import DecisionTreeClassifier


model = DecisionTreeClassifier(max_depth=14, random_state=42)
model.fit(X_train_scaled, y_train)
print('Train Accuracy', model.score(X_train_scaled, y_train))
print('Validation Accuracy', model.score(X_val_scaled, y_val))

ValueError: Number of labels=47520 does not match number of samples=38100

In [68]:
# # Model: Fit on train, score on val, predict on test
# model = LogisticRegression(solver='lbfgs', multi_class='auto', n_jobs=-1)
# model.fit(X_train_scaled, y_train)
# print('Validation Accuracy', model.score(X_val_scaled, y_val))
y_pred = model.predict(X_test_scaled)

# Write submission csv file
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-03.csv', index=False)

In [0]:
from google.colab import files
files.download('submission-03.csv')